# Computational Cognitive Neuroscience

## Assignment 2 

#### Douwe van Erp (s4258126) & Arianne Meijer - van de Griend (s4620135)

In [ ]:
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
import utils
import numpy as np
from chainer.cuda import to_cpu
from chainer.dataset import concat_examples
import matplotlib.pyplot as plt

### Definition of the different models

TODO explanation neural network

In [ ]:
class FCNN1(Chain):
    def __init__(self, n_out):
        super(FCNN1, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_out)

    def __call__(self, x):
        y = self.l1(x)
        return y

class FCNN2(Chain):
    def __init__(self, n_units, n_out):
        super(FCNN2, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)
            self.l2 = L.Linear(None, n_out)

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        y = self.l2(h1)
        return y

class FCNN3(Chain):
    def __init__(self, n_units, n_out):
        super(FCNN3, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_units)
            self.l2 = L.Linear(None, n_units)
            self.l3 = L.Linear(None, n_out)

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l2(h2)
        return y

TODO explanation convolutional network

In [ ]:
class CNN(Chain):
    def __init__(self, n_units, n_out):
        super(CNN, self).__init__()
        with self.init_scope():
            # 5 output channels, kernel size of 5, stride of 1, padding of 0
            self.l_conv = L.Convolution2D(None, 5, 5, 1, 0)
            self.l_fc = L.Linear(None, n_units)
            self.l_out = L.Linear(None, n_out)

    def __call__(self, x):
        h1 = F.max_pooling_2d(self.l_conv(x), 5, 1, 0)
        h2 = F.relu(self.l_fc(h1))
        y = self.l_out(h2)
        return y

TODO explanation extended network

In [ ]:
# Model iwth additional components (e.g. one of dropout, batch normalization, other activation functions, etc.).

### Performance deep neural networks
TODO visualise, interpret the results 

In [ ]:
def train_model(model):
    max_epoch = 20
    train_loss = []
    val_loss = []
    
    while train_iter.epoch < max_epoch:
    
        # Next minibatch
        train_batch = train_iter.next()
        image_train, target_train = concat_examples(train_batch)
    
        # Feedforward pass
        prediction_train = model(image_train)
    
        # Softmax cross entropy loss
        loss = F.softmax_cross_entropy(prediction_train, target_train)
    
        # Backpropagation
        model.cleargrads()
        loss.backward()
    
        # Update all the trainable paremters
        optimizer.update()
    
        # Check the validation accuracy of prediction after every epoch
        if train_iter.is_new_epoch:
    
            # Display the training loss
            print('epoch:{:02d} train_loss:{:.04f} '.format(train_iter.epoch, float(to_cpu(loss.data))), '')
    
            train_loss.append(float(to_cpu(loss.data)))
    
            test_losses = []
            test_accuracies = []
            while True:
                test_batch = test_iter.next()
                image_test, target_test = concat_examples(test_batch)
    
                # Forward pass
                prediction_test = model(image_test)
    
                # Calculate the loss
                loss_test = F.softmax_cross_entropy(prediction_test, target_test)
                test_losses.append(to_cpu(loss_test.data))
    
                # Calculate the accuracy
                accuracy = F.accuracy(prediction_test, target_test)
                accuracy.to_cpu()
                test_accuracies.append(accuracy.data)
    
                if test_iter.is_new_epoch:
                    test_iter.epoch = 0
                    test_iter.current_position = 0
                    test_iter.is_new_epoch = False
                    test_iter._pushed_position = None
                    break
    
            print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(np.mean(test_losses), np.mean(test_accuracies)))
            
            val_loss.append(np.mean(test_losses))
            
    return max_epoch, train_loss, val_loss

In [ ]:
def plot_training(epoch, train_loss, val_loss, title): 
    x = range(epoch)          
    fig = plt.figure()
    ax = plt.subplot(111)
    
    ax.plot(x, val_loss, 'r', label="Validation")
    ax.plot(x, train_loss, 'b', label="Training")
    
    plt.xlabel('Epoch')
    plt.xticks(range(epoch))
    plt.ylabel('Loss')
    plt.title(title)
    ax.legend()
    plt.show()

In [ ]:
# Load data and iterators
train, test = utils.get_mnist(n_train=100, n_test=100, n_dim=1, with_label=True)
train_iter = iterators.SerialIterator(train, batch_size=32, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size=32, repeat=False, shuffle=False)

model = FCNN1(10,10)
optimizer = optimizers.SGD()
optimizer.setup(model)

epoch, train_loss, val_loss = train_model(model)
plot_training(epoch, train_loss, val_loss, '1-Layer Fully Connected Neural Network')

TODO report conclusions

### Performance convolutional neural network
TODO visualise (plot loss), interpret the results + report conclusions

In [ ]:
# Load data and iterators
train, test = utils.get_mnist(n_train=100, n_test=100, n_dim=3, with_label=True)
train_iter = iterators.SerialIterator(train, batch_size=32, shuffle=True)
test_iter = iterators.SerialIterator(test, batch_size=32, repeat=False, shuffle=False)

model = CNN(64,10)
optimizer = optimizers.SGD()
optimizer.setup(model)

epoch, train_loss, val_loss = train_model(model)
plot_training(epoch, train_loss, val_loss, 'Convolutional Model')

TODO report conclusions

### Biological plausibility of convolution
TODO Explain in which ways convolution is biologically plausible and biologically implausible.

### Performance extended model
TODO visualise, interpret the results

In [ ]:
# Train test, visualise, etc code

TODO Report if your new architecture outperforms the original convnet architecture.
Provide a plot and a written explanation of your observed (better/worse) results.